In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import tempfile
import pandas as pd
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import dataset_schema
from tensorflow_transform.tf_metadata import dataset_metadata
import apache_beam as beam
import tensorflow_transform.beam as tft_beam
from tensorflow_transform.beam.tft_beam_io import transform_fn_io
from apache_beam.options.pipeline_options import PipelineOptions
import tensorflow_transform.beam.impl as beam_impl
 
from measurements import measure

tf.__version__

C:\Users\wgi\AppData\Local\anaconda3\envs\hsr3\lib\site-packages\apache_beam\__init__.py:84: UserWarning: Running the Apache Beam SDK on Python 3 is not yet fully supported. You may encounter buggy behavior or missing features.
  'Running the Apache Beam SDK on Python 3 is not yet fully supported. '


'1.13.1'

## Preprocessing measurement data for Machine Learning

A little trick: Take a note of the temporary directory containing the data for re-use in other notebooks.

In [3]:
temp_dir = tempfile.mkdtemp()
with open('temp_dir.txt', 'w') as file:
    file.write(temp_dir)
temp_dir

'/tmp/tmph9buhgd2'

In [4]:
# Signature data: Just the way it'll arrive at prediction time
signature_csv_train = os.path.join(temp_dir, "signature_train.csv")
signature_csv_eval = os.path.join(temp_dir, "signature_eval.csv")
signature_csv_test = os.path.join(temp_dir, "signature_test.csv")

# Training data: maybe scaled or further pre-processed.
training_csv = os.path.join(temp_dir, "training.csv")
eval_csv = os.path.join(temp_dir, "eval.csv")

# TFRecord: Allows for high performance input into computational graphs
training_tfr = os.path.join(temp_dir, "training.tfr")
eval_tfr = os.path.join(temp_dir, "eval.tfr")

print("You can find the files at:")
print(signature_csv_train, training_csv, training_tfr)

You can find the files at:
/tmp/tmph9buhgd2/signature_train.csv /tmp/tmph9buhgd2/training.csv /tmp/tmph9buhgd2/training.tfr


## The orginal data

In [5]:
data = measure(5)
data.to_csv(signature_csv_train, index=None)
data = pd.read_csv(signature_csv_train)
data.head()

,beta1,beta2,hour,humidity,weekday
0,-4.057947,-1.155510,5,13.045162,2
1,2.624675,-0.273144,13,25.166863,2
2,-4.986920,4.363070,4,7.379530,6
3,-2.007221,0.765549,9,14.036891,3
4,-0.759512,-4.683022,14,17.853140,1


### Specify the input and output formats

In [6]:
ORDERED_SIGNATURE_COLUMNS=["beta1", "beta2", "hour", "humidity", "weekday"]
header = bytes(",".join(ORDERED_SIGNATURE_COLUMNS), 'UTF-8')

In [7]:
feature_spec = {
    'beta1': tf.io.FixedLenFeature([1], tf.float32),
    'beta2': tf.io.FixedLenFeature([1], tf.float32),
    'weekday': tf.io.FixedLenFeature([1], tf.int64),
    'hour': tf.io.FixedLenFeature([1], tf.int64),
    'humidity': tf.io.FixedLenFeature([1], tf.float32)
}
schema = dataset_schema.from_feature_spec(feature_spec)

### Create an encoder and test it

In [8]:
csv_encoder = tft.coders.CsvCoder(ORDERED_SIGNATURE_COLUMNS, schema)
records = csv_encoder.decode("10.201, 10.101, 3,1.234,4")
print(records)
csv_encoder.encode(records)

{'beta1': array([10.201], dtype=float32), 'beta2': array([10.101], dtype=float32), 'hour': array([3]), 'humidity': array([1.234], dtype=float32), 'weekday': array([4])}


b'10.201,10.101,3,1.234,4'

---
# The Apache Beam pipeline 

In [9]:
def process_data(row):
    print(row)
    return row

### Dry run - Everything working?

In [10]:
ORDERED_SIGNATURE_COLUMNS

['beta1', 'beta2', 'hour', 'humidity', 'weekday']

In [11]:
with beam.Pipeline('DirectRunner', PipelineOptions()) as p:

    csv_encoder = tft.coders.CsvCoder(ORDERED_SIGNATURE_COLUMNS, schema)    

    _ = (p 
         | 'read_from_csv' >> beam.io.ReadFromText(
             file_pattern=signature_csv_train, coder=csv_encoder, skip_header_lines=1)
         
         | 'process_records' >> beam.Map(process_data)
         
         | 'write_to_csv' >> beam.io.WriteToText(
             file_path_prefix=training_csv, coder=csv_encoder, header=header)
        )


{'beta1': array([-4.0579476], dtype=float32), 'beta2': array([-1.1555097], dtype=float32), 'hour': array([5]), 'humidity': array([13.045162], dtype=float32), 'weekday': array([2])}
{'beta1': array([2.624675], dtype=float32), 'beta2': array([-0.2731437], dtype=float32), 'hour': array([13]), 'humidity': array([25.166862], dtype=float32), 'weekday': array([2])}
{'beta1': array([-4.98692], dtype=float32), 'beta2': array([4.3630695], dtype=float32), 'hour': array([4]), 'humidity': array([7.3795295], dtype=float32), 'weekday': array([6])}
{'beta1': array([-2.0072215], dtype=float32), 'beta2': array([0.7655485], dtype=float32), 'hour': array([9]), 'humidity': array([14.036891], dtype=float32), 'weekday': array([3])}
{'beta1': array([-0.75951177], dtype=float32), 'beta2': array([-4.6830215], dtype=float32), 'hour': array([14]), 'humidity': array([17.85314], dtype=float32), 'weekday': array([1])}


In [12]:
!echo "Reading from: " $training_csv-00000-of-00001
!cat $training_csv-00000-of-00001 | tail -10

Reading from:  /tmp/tmph9buhgd2/training.csv-00000-of-00001
beta1,beta2,hour,humidity,weekday
-4.0579476,-1.1555097,5,13.045162,2
2.624675,-0.2731437,13,25.166862,2
-4.98692,4.3630695,4,7.3795295,6
-2.0072215,0.7655485,9,14.036891,3
-0.75951177,-4.6830215,14,17.85314,1


### Serious transformation: Scale $\beta_1$ and $\beta_2$

In [13]:
for phase_csv in [signature_csv_train, signature_csv_eval, signature_csv_test]:
    measure(20000).to_csv(phase_csv, index=None)

In [14]:
def process_data(row):
    for c in ['beta1', 'beta2']:
        row[c] = tft.scale_to_0_1(row[c])
    return row

In [15]:
signature_metadata = dataset_metadata.DatasetMetadata(schema)

In [16]:
csv_encoder = tft.coders.CsvCoder(ORDERED_SIGNATURE_COLUMNS, schema)    
tfr_encoder = tft.coders.ExampleProtoCoder(schema)            

metadata_dir = os.path.join(temp_dir, "metadata")
with beam.Pipeline('DirectRunner', PipelineOptions()) as p:

    #
    # The context is provided for the AnalyseAndTransform step.
    # That step needs a hand to do its magic.
    #
    with tft_beam.Context(temp_dir=temp_dir):

        #
        # Read from csv, skip headers. Note that we use ordered columns in the encoder
        # 
        signature_data = ( p | 'read_from_csv' 
            >> beam.io.ReadFromText(
                 file_pattern=signature_csv_train, 
                coder=csv_encoder, skip_header_lines=1))

        #
        # attach the metadata: required for AnalyzeAndTransform
        #
        signature_data = ( signature_data, signature_metadata)

        #
        # Do the magic two steps and return also the transform-function
        #
        data_and_metadata, transform_fn = ( signature_data | "AnalyzeAndTransform" 
                         >> beam_impl.AnalyzeAndTransformDataset(process_data))
        
        #
        # split data and metadata
        #
        training_data, training_metadata = data_and_metadata

        #
        # Write the resulting data to a csv file
        #
        _ = (training_data | 'write_to_csv' 
             >> beam.io.WriteToText(
                 file_path_prefix=training_csv, coder=csv_encoder, header=header))

        #
        # For production purposes, we use the TFRecord format
        #
        _ = (training_data | 'write_to_tfr' 
             >> beam.io.WriteToTFRecord(
                 file_path_prefix=training_tfr, coder=tfr_encoder))

        
        #  Process evaluation data with the obtained transform_fn
        #
        signature_data = ( p | 'read_from_csv_eval' 
            >> beam.io.ReadFromText(
                 file_pattern=signature_csv_eval, coder=csv_encoder, skip_header_lines=1))

        signature_data = (signature_data, signature_metadata)

        # Use the transform_fn of the previous step here
        eval_data, _ = ((signature_data, transform_fn) 
                     | "TransformEval" >> tft_beam.TransformDataset())

        _ = (eval_data | 'write_to_tfr_eval' 
             >> beam.io.WriteToTFRecord(
                 file_path_prefix=eval_tfr, coder=tfr_encoder))

        
        
        #
        # Eventually, save the transform function for re-use at prediction time.
        #
        _ = (transform_fn | 'WriteTransformFn' 
             >> transform_fn_io.WriteTransformFn(metadata_dir))


Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


INFO:tensorflow:Assets added to graph.


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [17]:
!echo "Reading from: " $training_csv-00000-of-00001
!cat $training_csv-00000-of-00001 | tail -5
!echo metadata is here:
!ls $metadata_dir
!echo TFRecords are here: $training_tfr*

Reading from:  /tmp/tmph9buhgd2/training.csv-00000-of-00001
0.98481905,0.2808616,11,29.485474,1
0.6599091,0.8927997,21,21.971624,6
0.83055556,0.14907375,17,26.848066,6
0.34698424,0.27759552,2,17.552568,1
0.9909852,0.704738,5,26.602098,5
metadata is here:
transformed_metadata  transform_fn
TFRecords are here: /tmp/tmph9buhgd2/training.tfr-00000-of-00001


### Wrap up

We're now able to analyze and process any size of data, scale particular features to the interval $[0, 1]$ while saving the function that actually did it for later. We'll need that function to apply exactly the same scaling to the incoming data at prediction time. 
Note that by simply swapping ```'DirectRunner'``` in the Apache Beam pipeline by ```'DataFlowRunner'``` in an adequately configured GCP environment, we could have the pipeline executed on an arbitrarily large cluster.